## ストアドプロシージャの作成

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, upper, count, avg

In [ ]:
# データの読み込み
# クレンジング
# 簡単な集計
# 集計結果のテーブルへの書き込み

def sproc_etl_customer_summary(
    session:Session,
    raw_table: str,
    summary_table: str
):
    # 1)データの読み込み
    df_raw = session.table(raw_table)

    # 2) クレンジング
    df_clean = (df_raw.dropna().with_column("GENDER", upper(col("GENDER"))))

    # 3) 集計
    df_summary = (
        df_clean.group_by(col("GENDER"), col("CHURN"))
        .agg(
            count(col("ID")).alias("COUNT"),
            avg(col("AGE")).alias("AVG_AGE"),
            avg(col("ANNUAL_INCOME")).alias("AVG_INCOME")
        )
    )

    # 4)結果の上書き格納
    df_summary.write.mode("overwrite")\
        .save_as_table(summary_table)

    return "updated"



In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.types import StringType

session = get_active_session()

session.sproc.register(
    func = sproc_etl_customer_summary,
    name = "SP_ETL_CUSTOMER_SUMMARY",
    package = ["snowflake-snowpark-python"],
    input_types = [StringType(), StringType()],
    return_type = StringType(),
    replace = True,
)

In [ ]:
CREATE OR REPLACE TABLE CUSTOMER_SUMMARY(
    GENDER STRING,
    CHURN NUMBER,
    COUNT NUMBER,
    AVG_AGE FLOAT,
    AVG_INCOME FLOAT
);

In [ ]:
-- ストアドプロシージャの呼び出し
CALL SP_ETL_CUSTOMER_SUMMARY(
    'CUSTOMER_DATA_1000',
    'CUSTOMER_SUMMARY'
)

In [ ]:
select * from CUSTOMER_SUMMARY;